In [1]:
using Random
using Distributions
using FLoops
using Base.Threads
using BenchmarkTools

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

8

In [2]:
dtype = Float32
seed = 123

# DATA CONFIG

alpha = 0.05

# data
B  = 100   # num. coverage probabilities per boxplot
S  = 4300  # num. samples per coverage probability
nx = 100   # size of group 1
ny = 50    # size of group 2
nperm = try
    binomial(nx+ny, nx)
catch OverflowError
    Inf
end
    
if nperm > 30_000
    Random.seed!(123)
    px, py = partition(nx, ny, 10_000)
else
    px, py = partition(nx, ny)
end

bits = vcat(ones(Int, nx), zeros(Int, ny))
addx, addy = bits[px], bits[py]
px[:, 1:5]

100×5 Matrix{Int64}:
   1   79   71   14   60
   2   89   99  143   84
   3  134   98   97   34
   4   32  106  142   31
   5   81   65  109  100
   6   62   83   27   94
   7   13  115  144  135
   8  141  121   56   82
   9   91  145   98   22
  10   56  120  138   87
  11   84   30   90   58
  12  128   97  150   66
  13   18   31  129  130
   ⋮                 
  89  127    3   65   83
  90  110  102  126   75
  91  133   19   23   28
  92  144    8   44   99
  93   87  114  133   37
  94   98   41   60    8
  95  103  107  105   38
  96  105  143  131   11
  97   75   43   20   59
  98   82   16   21  140
  99   65  119   37    7
 100   46    9   55   50

In [13]:
# POPULATION SETTINGS

Random.seed!(123)

distrTypeX = Laplace{dtype}
X_mu = random(Uniform(0, 1), B)
X_sigma = random(Uniform(2, 4), B)
distrX = map(distrTypeX, X_mu, X_sigma)

distrTypeY = Laplace{dtype}
Y_mu = X_mu .+ random(Uniform(-5, 5), B)
Y_sigma = X_sigma
distrY = map(distrTypeY, Y_mu, Y_sigma)
;

In [14]:
deltas = @. mean(distrX) - mean(distrY)

@show distrX[1:2]
@show distrY[1:2]
@show deltas[1:2];

distrX[1:2] = Laplace{Float32}[Laplace{Float32}(μ=0.906f0, θ=3.908f0), Laplace{Float32}(μ=0.443f0, θ=3.692f0)]
distrY[1:2] = Laplace{Float32}[Laplace{Float32}(μ=5.556f0, θ=3.908f0), Laplace{Float32}(μ=-3.8500001f0, θ=3.692f0)]
deltas[1:2] = Float32[-4.65, 4.293]


In [15]:
Random.seed!(123)
xs = [dtype.(rand(distrX[i], nx, S)) for i in 1:B]
ys = [dtype.(rand(distrY[i], ny, S)) for i in 1:B]
@show size(ys)
ys[2]

size(ys) = (100,)


50×4300 Matrix{Float32}:
  -0.333925     1.31092    -7.38782   …   -3.37927   -1.1654     2.74575
   0.0676902   -5.67038    -7.46346       -1.27034   -5.84355   -3.15161
  -1.69411     -8.48162    -8.91671       -3.78911    5.17836   -8.30319
  -9.54606     -3.3755     -5.09105       -5.12691   -4.72239   -6.01472
  -3.9664       9.08281    -4.48176      -16.6926     3.44047   -2.896
  -4.14218     -0.422397   -3.39665   …   -8.43455   -2.42219   -3.20621
  -6.83521     -3.30915    -9.02733       -3.53746   -1.23077   -7.15858
  -4.50355      0.364631   -2.69697       -1.85944    6.81848   -7.10866
 -19.2384      -3.64316    -7.09127       -1.41352   -9.70076   -5.54769
  -3.23951     -6.43108   -14.0481        -4.65282   -5.84959   -4.77871
   1.67845     -9.69094    -5.75341   …   -0.372413  -4.95373   -3.99881
  -1.72792     21.2682     -2.96603       -5.83838   -2.44691   -1.98653
  -8.47004      1.39169    -2.55322        7.12183    3.95665    4.44845
   ⋮                        

In [16]:
# flatten into 3D matrix
X = reshape(hcat(xs...), nx, S, B)
Y = reshape(hcat(ys...), ny, S, B)
Y[:,:,2]

50×4300 Matrix{Float32}:
  -0.333925     1.31092    -7.38782   …   -3.37927   -1.1654     2.74575
   0.0676902   -5.67038    -7.46346       -1.27034   -5.84355   -3.15161
  -1.69411     -8.48162    -8.91671       -3.78911    5.17836   -8.30319
  -9.54606     -3.3755     -5.09105       -5.12691   -4.72239   -6.01472
  -3.9664       9.08281    -4.48176      -16.6926     3.44047   -2.896
  -4.14218     -0.422397   -3.39665   …   -8.43455   -2.42219   -3.20621
  -6.83521     -3.30915    -9.02733       -3.53746   -1.23077   -7.15858
  -4.50355      0.364631   -2.69697       -1.85944    6.81848   -7.10866
 -19.2384      -3.64316    -7.09127       -1.41352   -9.70076   -5.54769
  -3.23951     -6.43108   -14.0481        -4.65282   -5.84959   -4.77871
   1.67845     -9.69094    -5.75341   …   -0.372413  -4.95373   -3.99881
  -1.72792     21.2682     -2.96603       -5.83838   -2.44691   -1.98653
  -8.47004      1.39169    -2.55322        7.12183    3.95665    4.44845
   ⋮                        

In [17]:
function save_ci_results(results, methodId, B, S, pooled=nothing, two_sided=nothing; prefix="", dir="./")
    averages = []

    for batchId in 1:B
        batch = results[methodId, batchId, :]
        coverage = sum([j for (j, _) in batch]) / S
        width = sum([j for (_, j) in batch]) / S
        push!(averages, (coverage, width))
    end
    if isnothing(two_sided)
        save(averages, distrX[1:B], distrY[1:B], alpha, prefix=prefix, dir=dir)
    else
        alpha_ = two_sided ? alpha : alpha / 2
        save(averages, distrX[1:B], distrY[1:B], alpha_, pooled, two_sided, prefix=prefix, dir=dir)
    end
end

function save_permutation_results(results, B, S; prefix="", dir="./")
    i = 1
    per_method = []
    for two_sided in [true, false]
        for pooled in [true, false]
            if i in [2, 4]  # use only unpooled for now
                save_ci_results(results, i, B, S, pooled, two_sided, prefix=prefix, dir=dir)
            end
            i += 1
        end
    end
end

save_permutation_results (generic function with 1 method)

In [18]:
function cache(groups, masks)
    mean_og = mean(groups, dims=1)
    var_og = var(groups, dims=1)
    nshift = sum(masks, dims=1)
    shift_sum = sum(groups .* masks, dims=1)
    return P(mean_og, var_og, nshift, shift_sum, size(groups, 1))
end

cache (generic function with 1 method)

In [19]:
x = X[:,1,1]
y = Y[:,1,1]
pooled = vcat(x, y)
xs = pooled[px]
ys = pooled[py]
xcache = cache(xs, addx)
ycache = cache(ys, addy)

wide, narrow = t_estimates(x, y, false)
@time permInterval(xcache, ycache, wide, narrow, false, alpha, twoSided, twoSided, 0.005)

  0.009405 seconds (1.78 k allocations: 11.905 MiB)


(-7.9077641749761405, -4.167715332470147)

In [ ]:
T = Threads.nthreads()
results = Array{Union{Tuple, Nothing}, 3}(nothing, 6, B, S)

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:B, s in 1:S
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]
    
#     pooled = vcat(x, y)
#     xs = @inbounds pooled[px]
#     ys = @inbounds pooled[py]
#     xcache = cache(xs, addx)
#     ycache = cache(ys, addy)
#     wide, narrow = t_estimates(x, y, false)
 
#     results[1, b, s] = permInterval(xcache, ycache, deltas[b], true, alpha, twoSided, twoSided, 0.0005)    
#     results[2, b, s] = permInterval(xcache, ycache, wide, narrow, deltas[b], false, alpha, twoSided, twoSided, 0.0005)
#     results[3, b, s] = permInterval(xcache, ycache, deltas[b], true, alpha/2, greater, smaller, 0.0005)
#     results[4, b, s] = permInterval(xcache, ycache, wide, narrow, deltas[b], false, alpha/2, greater, smaller, 0.0005)
    
    #"""
    results[5, b, s] = bootstrap(x, y, deltas[b], alpha, nsamples=10_000)
    results[6, b, s] = tconf(x, y, deltas[b], alpha, false)
    #"""
end

dir = "../results/" * string(nx) * "_" * string(ny) * "/4/"
# save_permutation_results(results, B, S; dir=dir)
save_ci_results(results, 5, B, S; prefix="bs", dir=dir)
save_ci_results(results, 6, B, S; prefix="t", dir=dir)

In [ ]:
b = 10
s = 100
[sum(x for (x, _) in results[4, b, 1:s]) / s for b in 1:b]

10-element Vector{Float64}:
 0.95
 0.98
 0.95
 0.94
 0.95
 0.93
 0.95
 0.92
 0.94
 0.97